<a href="https://colab.research.google.com/github/ucaokylong/LLM_learning/blob/main/Fine_tune_Qwen2_5_Coder_(32B)_Conversational.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd ./drive/MyDrive/qwen_project/new_exp

/content/drive/MyDrive/qwen_project/new_exp


**Read our [Gemma 3 blog](https://unsloth.ai/blog/gemma3) for what's new in Unsloth and our [Reasoning blog](https://unsloth.ai/blog/r1-reasoning) on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

qwen_models = [
    "unsloth/Qwen2.5-Coder-32B-Instruct",      # Qwen 2.5 Coder 2x faster
    "unsloth/Qwen2.5-Coder-7B",
    "unsloth/Qwen2.5-14B-Instruct",            # 14B fits in a 16GB card
    "unsloth/Qwen2.5-7B",
    "unsloth/Qwen2.5-72B-Instruct",            # 72B fits in a 48GB card
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-Coder-32B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: We'll be using `/tmp/unsloth_compiled_cache` for temporary Unsloth patches.
Standard import failed for UnslothBCOTrainer: No module named 'UnslothBCOTrainer'. Using tempfile instead!
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.50.0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/280k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.19 patched 64 layers with 64 QKV layers, 64 O layers and 64 MLP layers.


In [ ]:
import re
import pandas as pd
from datasets import Dataset
import torch
import random

In [ ]:
def detect_question_type(question):
    """Classify questions into 3 types using advanced pattern matching"""
    question_lower = question.lower()

    # Detect code questions with programming keywords
    code_pattern = r'\b(code|program|algorithm|syntax|debug|compile|runtime|script|function)\b'
    if re.search(code_pattern, question_lower):
        return "code"

    # Detect matching questions with structural patterns
    matching_pattern = r'\b(match|pair|group\s+1|column\s+a|connect|association)\b|[-–]'
    if re.search(matching_pattern, question_lower):
        return "matching"

    # Default to general knowledge questions
    return "general"

def format_code_prompt(question, choices):
    """Generate chain-of-thought prompt for programming questions"""
    return f"""**Programming Problem Analysis**

Question:
{question}

Reasoning Steps:
1. Analyze the code structure and identify key components
2. Determine the core problem or error in the code
3. Evaluate each option for technical correctness
4. Consider edge cases and potential exceptions
5. Select the most logically consistent solution

Options:
""" + '\n'.join([f'{chr(65+i)}. {choice}' for i, choice in enumerate(choices)]) + "\nAnswer:"

def format_matching_prompt(question, choices):
    """Generate analytical prompt for matching questions"""
    return f"""**Matching Analysis**

Question:
{question}

Analysis Process:
1. Identify key characteristics of each item in both groups
2. Establish logical relationships between items
3. Verify one-to-one correspondence
4. Check for consistency in matching patterns
5. Eliminate contradictory pairings

Options:
""" + '\n'.join([f'{chr(65+i)}. {choice}' for i, choice in enumerate(choices)]) + "\nAnswer:"

def format_general_prompt(question, choices):
    """Generate reasoning prompt for knowledge-based questions"""
    return f"""**Logical Reasoning Process**

Question:
{question}

Critical Thinking Steps:
1. Identify the core concept being tested
2. Recall relevant theories/principles
3. Analyze each option systematically
4. Eliminate options with factual errors
5. Compare remaining alternatives

Options:
""" + '\n'.join([f'{chr(65+i)}. {choice}' for i, choice in enumerate(choices)]) + "\nAnswer:"


In [ ]:
def create_conversations(row):
    """Create structured conversations for model training"""
    question_type = detect_question_type(row['question'])
    choices = eval(row['choices'])  # Convert string to list

    prompt_functions = {
        "code": format_code_prompt,
        "matching": format_matching_prompt,
        "general": format_general_prompt
    }

    user_content = prompt_functions[question_type](row['question'], choices)

    return {
        "conversations": [
            {"role": "user", "content": user_content},
            {"role": "assistant", "content": row['answer']}
        ]
    }

In [ ]:
# Load and clean data
df = pd.read_csv('../b6_train_data.csv').dropna(subset=['answer'])
dataset = Dataset.from_pandas(df)
dataset = dataset.map(create_conversations)

Map:   0%|          | 0/3949 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

In [ ]:
from unsloth.chat_templates import get_chat_template, standardize_sharegpt

# Áp dụng template cho tokenizer
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)

# Hàm chuyển đổi conversations thành text
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

# Chuẩn hóa dataset và áp dụng formatting
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched=True)

Unsloth: Standardizing formats (num_proc=12):   0%|          | 0/3949 [00:00<?, ? examples/s]

Map:   0%|          | 0/3949 [00:00<?, ? examples/s]

In [ ]:
dataset[0]['text']

'<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\n**Programming Problem Analysis**\n\nQuestion:\nQuestion: What will be output of the following code?\n#include<stdio.h>\nint main()\n{\n    printf("%d\\t",sizeof(6.5));\n    printf("%d\\t",sizeof(90000));\n    printf("%d",sizeof(\'A\'));\n    return 0;\n}\n[\'8 4 2\', \'8 4 2\', \'8 4 4\', \'8 4 3\']\n\nAnswer: \n\nReasoning Steps:\n1. Analyze the code structure and identify key components\n2. Determine the core problem or error in the code\n3. Evaluate each option for technical correctness\n4. Consider edge cases and potential exceptions\n5. Select the most logically consistent solution\n\nOptions:\nA. 8 4 2\nB. 8 4 2\nC. 8 4 4\nD. 8 4 3\nAnswer:<|im_end|>\n<|im_start|>assistant\nC<|im_end|>\n'

In [ ]:
# Training Parameters
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 3,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        optim = "paged_adamw_32bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        output_dir = "outputs",
        logging_steps = 20,
        seed = 42,
        report_to = "none"
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/3949 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,949 | Num Epochs = 3 | Total steps = 1,479
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 134,217,728/32,000,000,000 (0.42% trained)


Step,Training Loss
20,2.047300
40,1.428100
60,0.779600
80,0.490100
100,0.415100
120,0.363400
140,0.345700
160,0.324600
180,0.333200
200,0.347500


In [ ]:
def extract_answer(response):
    """Robust answer extraction with validation"""
    # Find all capital letters in specific patterns
    patterns = [
        r"Final Answer:\s*([A-D])",
        r"Correct Answer:\s*([A-D])",
        r"^Answer:\s*([A-D])",
        r"\b([A-D])\b(?!\.\d)",  # Exclude version numbers like "A.1"
    ]

    for pattern in patterns:
        match = re.search(pattern, response, re.IGNORECASE | re.MULTILINE)
        if match:
            return match.group(1).upper()

    # Fallback: Last occurrence method
    last_choice = re.findall(r'\b[A-D]\b', response.upper())
    return last_choice[-1] if last_choice else random.choice(['A', 'B', 'C', 'D'])

def predict_answer(question, choices):
    """End-to-end prediction pipeline"""
    question_type = detect_question_type(question)
    choices = eval(choices) if isinstance(choices, str) else choices

    prompt = {
        "code": format_code_prompt,
        "matching": format_matching_prompt,
        "general": format_general_prompt
    }[question_type](question, choices)

    messages = [{"role": "user", "content": prompt}]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True,
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(
        inputs,
        max_new_tokens = 512,
        temperature = 0.8,
        top_k = 50,
        top_p = 0.95,
        repetition_penalty = 1.1,
        do_sample = True,
    )

    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return extract_answer(full_response.split("Answer:")[-1])

# ================== Generate Predictions ==================
test_df = pd.read_csv('../b6_test_data.csv')
predictions = []

for _, row in test_df.iterrows():
    try:
        answer = predict_answer(row['question'], row['choices'])
    except Exception as e:
        print(f"Error processing task {row['task_id']}: {str(e)}")
        answer = random.choice(['A', 'B', 'C', 'D'])

    predictions.append({
        'task_id': row['task_id'],
        'answer': answer
    })

pd.DataFrame(predictions).to_csv('./submission.csv', index=False)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
